In [ ]:
#CORRECT VERSION 2

import sqlite3
from pathlib import Path
import pandas as pd

class REITDatabase:
    def __init__(self, db_path='version2/data/database/reit_scanner.db'):
        self.db_path = Path(db_path)
        self.db_path.parent.mkdir(parents=True, exist_ok=True)
        self.conn = None
        print(f"Database will be created at: {self.db_path.resolve()}")
    def connect(self):
        self.conn = sqlite3.connect(self.db_path)
        return self.conn
    
    def create_tables(self):
        """Create all necessary tables"""
        conn = self.connect()
        cursor = conn.cursor()
        
        # Table 1: REIT Universe
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS reits (
                ticker TEXT PRIMARY KEY,
                company_name TEXT,
                cik TEXT,
                market_cap REAL,
                sector TEXT,
                reit_type TEXT,  -- Equity, Mortgage, Hybrid
                geography TEXT,
                is_active INTEGER DEFAULT 1,
                date_added TEXT,
                last_updated TEXT
            )
        ''')
        
        # Table 2: SEC Filings Metadata
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS filings (
                filing_id INTEGER PRIMARY KEY AUTOINCREMENT,
                ticker TEXT,
                filing_type TEXT,  -- 10-K, 10-Q
                filing_date TEXT,
                period_end_date TEXT,
                accession_number TEXT UNIQUE,
                file_path TEXT,
                file_url TEXT,
                download_date TEXT,
                processed INTEGER DEFAULT 0,
                FOREIGN KEY (ticker) REFERENCES reits(ticker)
            )
        ''')
        
        # Table 3: Financial Metrics
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS financials (
                metric_id INTEGER PRIMARY KEY AUTOINCREMENT,
                ticker TEXT,
                filing_id INTEGER,
                period_end_date TEXT,
                
                -- Balance Sheet
                total_assets REAL,
                total_debt REAL,
                current_debt REAL,
                long_term_debt REAL,
                shareholders_equity REAL,
                cash_and_equivalents REAL,
                total_liabilities REAL,
                
                -- Income Statement
                total_revenue REAL,
                operating_income REAL,
                interest_expense REAL,
                net_income REAL,
                ebitda REAL,
                
                -- Cash Flow
                operating_cash_flow REAL,
                
                -- Calculated Ratios
                debt_to_assets REAL,
                debt_to_equity REAL,
                interest_coverage REAL,
                current_ratio REAL,
                
                extraction_date TEXT,
                FOREIGN KEY (ticker) REFERENCES reits(ticker),
                FOREIGN KEY (filing_id) REFERENCES filings(filing_id)
            )
        ''')
        
        # Table 4: NLP Analysis Results
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS nlp_analysis (
                analysis_id INTEGER PRIMARY KEY AUTOINCREMENT,
                ticker TEXT,
                filing_id INTEGER,
                distress_score REAL,  -- 0-10
                sentiment_score REAL,  -- -1 to 1
                analysis_date TEXT,
                model_used TEXT,
                FOREIGN KEY (ticker) REFERENCES reits(ticker),
                FOREIGN KEY (filing_id) REFERENCES filings(filing_id)
            )
        ''')
        
        # Table 5: Distress Flags
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS distress_flags (
                flag_id INTEGER PRIMARY KEY AUTOINCREMENT,
                ticker TEXT,
                filing_id INTEGER,
                flag_category TEXT,  -- covenant, liquidity, asset_sale, etc.
                severity TEXT,  -- low, medium, high, critical
                description TEXT,
                supporting_quote TEXT,
                page_reference TEXT,
                detected_date TEXT,
                FOREIGN KEY (ticker) REFERENCES reits(ticker),
                FOREIGN KEY (filing_id) REFERENCES filings(filing_id)
            )
        ''')


                
        # Table 6: Final Scores
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS reit_scores (
                score_id INTEGER PRIMARY KEY AUTOINCREMENT,
                ticker TEXT,
                analysis_date TEXT,
                period_end_date TEXT,
                
                leverage_score REAL,
                liquidity_score REAL,
                distress_score REAL,
                final_score REAL,
                rank INTEGER,
                
                FOREIGN KEY (ticker) REFERENCES reits(ticker),
                UNIQUE(ticker, analysis_date)
            )
        ''')
        
        conn.commit()
        print("✓ Database tables created successfully")
        
    def close(self):
        if self.conn:
            self.conn.close()


# Usage
if __name__ == "__main__":
    db = REITDatabase()
    db.create_tables()
    db.close()


Database will be created at: C:\Users\0wner\Documents\testing colin\version2\data\database\reit_scanner.db
✓ Database tables created successfully


In [2]:
import yfinance as yf
import pandas as pd
import requests
from datetime import datetime
import time
from pathlib import Path
import sys
sys.path.append('src')
# Use the existing REITDatabase instance if available
if 'db' in globals() and isinstance(db, REITDatabase):
    REITDatabase = type(db)

class REITUniverseBuilder:
    def __init__(self):
        self.db = REITDatabase()
        self.reits = []
        
    def get_reit_tickers_from_sec(self):
        """
        Get REITs from SEC using EDGAR API
        SIC Code 6798 = Real Estate Investment Trusts
        """
        print("Fetching REIT list from SEC...")
        
        # SEC Company Tickers JSON endpoint
        url = "https://www.sec.gov/files/company_tickers.json"
        
        headers = {
            'User-Agent': 'REIT Scanner your.email@example.com'  # Replace with your email
        }
        
        response = requests.get(url, headers=headers)
        data = response.json()
        
        # Convert to DataFrame - data is dict with numeric keys
        companies = pd.DataFrame.from_dict(data, orient='index')
        
        # Columns should now be: cik_str, ticker, title
        # Let's verify and clean
        print(f"Columns found: {companies.columns.tolist()}")
        print(f"Total companies: {len(companies)}")
        
        # Filter for REITs (SIC code 6798 and 6799)
        # Note: The JSON might not include SIC codes directly
        # We'll need to check what's actually available
        print("\nFirst few rows:")
        print(companies.head())
        
        # If SIC code isn't in the JSON, we'll need a different approach
        tickers_list = ['AFCG','AHR','AOMR','BHM','BNL','CDP','CLPR','CMTG','EFC',
                   'ELME','ESBA','FBRT','GIPR','GPMT','IHT','LAND','LFT','LINE',
                   'LOAN','MDV','MITT','NLOP','NXDT','NYMT','ONL','OPI','PKST','PW','REFI',
                   'RITM','ROIC','SACH','SILA','SQFT','STRW','SUNS','UNIT','WHLR','WSR','AAT',
                   'ABR','ACR','ACRE','ADC','AGNC','AHH','AHT','AIRC','AIV','AJX','AKR','ALEX',
                   'ALX','AMH','AMT','APLE','ARE','ARI','ARR','AVB','BDN','BFS','BHR','BRSP','BRT',
                   'BRX','BXMT','BXP','CBL','CCI','CDR','CHCT','CHMI','CIM','CIO','CLDT','CMCT',
                   'CMO','COLD','CONE','CORR','CPLG','CPT','CSR','CTO','CTRE','CTT','CUBE','CUZ',
                   'CXP','CXW','DBRG','DCT','DEA','DEI','DHC','DLR','DOC','DRH','DX','EARN','EGP',
                   'ELS','EPR','EPRT','EQIX','EQR','ESRT','ESS','EXR','FCPT','FPI','FR','FREVS',
                   'FRT','FSP','GLPI','GMRE','GNL','GOOD','GTY','HASI','HIW','HPP','HR','HST','IIPR',
                   'ILPT','INN','INVH','IRM','IRT','IVR','IVT','JBGS','KIM','KRC','KREF','KRG',
                   'LADR','LAMR','LTC','LXP','MAA','MAC','MDRR','MFA','MPW','NHI','NLY','NNN',
                   'NREF','NSA','NTST','NXRT','NYC','O','OHI','OLP','ORC','OUT','PCH','PDM','PEB',
                   'PECO','PGRE','PINE','PK','PLD','PLYM','PMT','PSA','PSTL','RC','REG','REXR','RHP',
                   'RLJ','RPAI','RWT','RYN','SAFE','SBAC','SBRA','SELF','SEVN','SHO','SITC','SKT',
                   'SLG','SOHO','SPG','SRG','STAG','STWD','SUI','SVC','TCI','TRNO','TRTX','TWO',
                   'UDR','UE','UHT','UMH','VER','VICI','VNO','VRE','VTR','WELL','WPC','WRE','WRI',
                   'WY','XHR'
]
            
        mask = companies['ticker'].isin(tickers_list)
        reits = companies[mask].copy()
        
        print(f"Found {len(reits)} potential REITs")
        return reits
    
    def enrich_with_market_data(self, reit_df):
        """Add market cap and verify these are tradable stocks"""
        enriched_data = []
        
        # Use enumerate so the progress counter is always an int
        for i, (_, row) in enumerate(reit_df.iterrows(), start=1):
            ticker = row['ticker']
            print(f"Processing {ticker}... ({i}/{len(reit_df)})")
        
            try:
                stock = yf.Ticker(ticker)
                info = stock.info
                
                # Get basic data
                data = {
                    'ticker': ticker,
                    'company_name': row['title'],
                    'cik': str(row['cik_str']).zfill(10),
                    'market_cap': info.get('marketCap', 0),
                    'sector': info.get('sector', 'Unknown'),
                    'reit_type': self._classify_reit_type(row['title']),
                    'geography': 'US',  # Can enhance this later
                    'is_active': 1 if info.get('marketCap', 0) > 0 else 0,
                    'date_added': datetime.now().strftime('%Y-%m-%d'),
                    'last_updated': datetime.now().strftime('%Y-%m-%d')
                }
                
                enriched_data.append(data)
                time.sleep(0.5)  # Be nice to Yahoo Finance
                
            except Exception as e:
                print(f"  ⚠ Error processing {ticker}: {e}")
                continue
        
        return pd.DataFrame(enriched_data)
    
    def _classify_reit_type(self, company_name):
        """Simple classification based on name"""
        name_lower = company_name.lower()
        
        if any(word in name_lower for word in ['mortgage', 'mbs', 'loan']):
            return 'Mortgage'
        elif 'hybrid' in name_lower:
            return 'Hybrid'
        else:
            return 'Equity'

    def filter_universe(self, df, max_market_cap=1_000_000_000):
        """Apply filters to get relevant REITs"""
        filtered = df[
            (df['market_cap'] <= max_market_cap) &
            (df['is_active'] == 1)
        ].copy()
        
        print(f"\nFiltered from {len(df)} to {len(filtered)} REITs")
        print(f"  - Max market cap: ${max_market_cap:,.0f}")
        print(f"  - Active trading only")
        
        return filtered

    
    def save_to_database(self, df):
        """Save REIT universe to database"""
        conn = self.db.connect()
        
        df.to_sql('reits', conn, if_exists='replace', index=False)
        
        print(f"\n✓ Saved {len(df)} REITs to database")
        
        # Show summary
        print("\nREIT Universe Summary:")
        print(df.groupby('reit_type').size())
        print(f"\nTop 10 by Market Cap:")
        print(df.nlargest(10, 'market_cap')[['ticker', 'company_name', 'market_cap']])
        
        self.db.close()
        
        # Also save to CSV for easy viewing
        output_path = Path('version2/data/processed/reit_universe.csv')
        output_path.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(output_path, index=False)
        print(f"\n✓ Also saved to {output_path}")
    
    def build(self, max_market_cap=1_000_000_000):
        """Main method to build the universe"""
        print("="*60)
        print("BUILDING REIT UNIVERSE")
        print("="*60)
        
        # Step 1: Get REITs from SEC
        sec_reits = self.get_reit_tickers_from_sec()
        
        # Step 2: Enrich with market data
        print("\nEnriching with market data...")
        enriched_reits = self.enrich_with_market_data(sec_reits)
        
        # Step 3: Filter
        print("\nApplying filters...")
        filtered_reits = self.filter_universe(enriched_reits, max_market_cap)
        
        # Step 4: Save
        self.save_to_database(filtered_reits)
        
        return filtered_reits


if __name__ == "__main__":
    builder = REITUniverseBuilder()
    reits = builder.build(max_market_cap=1_000_000_000)

Database will be created at: C:\Users\0wner\Documents\testing colin\version2\data\database\reit_scanner.db
BUILDING REIT UNIVERSE
Fetching REIT list from SEC...
Columns found: ['cik_str', 'ticker', 'title']
Total companies: 10303

First few rows:
   cik_str ticker           title
0  1045810   NVDA     NVIDIA CORP
1  1652044  GOOGL   Alphabet Inc.
2   320193   AAPL      Apple Inc.
3   789019   MSFT  MICROSOFT CORP
4  1018724   AMZN  AMAZON COM INC
Found 198 potential REITs

Enriching with market data...
Processing WELL... (1/198)
Processing PLD... (2/198)
Processing AMT... (3/198)
Processing EQIX... (4/198)
Processing SPG... (5/198)
Processing O... (6/198)
Processing DLR... (7/198)
Processing PSA... (8/198)
Processing CCI... (9/198)
Processing VTR... (10/198)
Processing EXR... (11/198)
Processing VICI... (12/198)
Processing AVB... (13/198)
Processing IRM... (14/198)
Processing EQR... (15/198)
Processing SBAC... (16/198)
Processing ESS... (17/198)
Processing WY... (18/198)
Processing MAA

In [9]:
import requests
import time
from pathlib import Path
from datetime import datetime
import pandas as pd
import sqlite3
from bs4 import BeautifulSoup

class SECFilingDownloader:
    def __init__(self, db_path='version2/data/database/reit_scanner.db'):
        self.db_path = db_path
        self.headers = {
            'User-Agent': 'REIT Scanner tennesseesamwright999@gmail.com',  # Replace with your email
            'Accept-Encoding': 'gzip, deflate',
        }
        self.base_url = "https://www.sec.gov"
        
    def get_reit_list(self):
        """Get list of REITs from database"""
        conn = sqlite3.connect(self.db_path)
        df = pd.read_sql("SELECT ticker, cik, company_name FROM reits WHERE is_active = 1", conn)
        conn.close()
        return df
    
    def get_filing_documents(self, accession_number, cik):
        """
        Get the actual document URL (not the viewer wrapper)
        """
        # Clean accession number (remove dashes)
        acc_clean = accession_number.replace('-', '')
        cik_clean = str(cik).replace('-', '').zfill(10)
        
        # This is the filing detail page
        index_url = f"{self.base_url}/cgi-bin/viewer?action=view&cik={cik_clean}&accession_number={accession_number}&xbrl_type=v"
        
        try:
            response = requests.get(index_url, headers=self.headers)
            response.raise_for_status()
            time.sleep(0.15)
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Find the table with document links
            table = soup.find('table', {'summary': 'Document Format Files'})
            if not table:
                table = soup.find('table', {'class': 'tableFile'})
            
            if table:
                rows = table.find_all('tr')
                for row in rows:
                    cells = row.find_all('td')
                    if len(cells) >= 3:
                        # Look for the main document (10-K or 10-Q)
                        doc_type = cells[3].text.strip() if len(cells) > 3 else ''
                        
                        if '10-K' in doc_type or '10-Q' in doc_type:
                            # Get the document link
                            link = cells[2].find('a')
                            if link and link.get('href'):
                                doc_url = link['href']
                                # Make sure it's not the viewer wrapper
                                if not 'ix?doc=' in doc_url:
                                    if not doc_url.startswith('http'):
                                        doc_url = f"{self.base_url}{doc_url}"
                                    return doc_url
            
            # Fallback: try direct URL construction
            # Pattern: /Archives/edgar/version2/data/{cik}/{accession-no-dashes}/{filename}
            fallback_url = f"{self.base_url}/Archives/edgar/data/{cik_clean.lstrip('0')}/{acc_clean}/{accession_number}.txt"
            return fallback_url
            
        except Exception as e:
            print(f"  ⚠ Error getting document URL: {e}")
            return None
    
    def download_filing(self, doc_url, ticker, accession_number, filing_type):
        """Download the actual filing document"""
        try:
            response = requests.get(doc_url, headers=self.headers)
            response.raise_for_status()
            time.sleep(0.15)
            
            # Save to file
            save_path = Path(f'version2/data/raw/filings/{ticker}')
            save_path.mkdir(parents=True, exist_ok=True)
            
            filename = f"{accession_number.replace('-', '')}_{filing_type.replace('/', '_')}.html"
            file_path = save_path / filename
            
            with open(file_path, 'wb') as f:
                f.write(response.content)
            
            return str(file_path)
            
        except Exception as e:
            print(f"  ⚠ Error downloading: {e}")
            return None
    
    def get_company_filings_metadata(self, cik, filing_type, count=2):
        """Get filing metadata from SEC"""
        cik_clean = str(cik).replace('-', '').zfill(10)
        
        url = f"{self.base_url}/cgi-bin/browse-edgar"
        params = {
            'action': 'getcompany',
            'CIK': cik_clean,
            'type': filing_type,
            'dateb': '',
            'owner': 'exclude',
            'count': count,
            'output': 'atom'
        }
        
        try:
            response = requests.get(url, params=params, headers=self.headers)
            response.raise_for_status()
            time.sleep(0.15)
            
            soup = BeautifulSoup(response.content, 'xml')
            entries = soup.find_all('entry')
            
            filings = []
            for entry in entries:
                accession = entry.find('accession-number')
                filing_date = entry.find('filing-date')
                
                if accession and filing_date:
                    filings.append({
                        'accession_number': accession.text,
                        'filing_date': filing_date.text,
                        'filing_type': filing_type
                    })
            
            return filings
            
        except Exception as e:
            print(f"  ⚠ Error fetching metadata: {e}")
            return []
    
  
    def save_filing_to_db(self, ticker, filing_type, filing_date, accession_number, file_path, file_url):
        """Save filing metadata to database"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        try:
            cursor.execute('''
                INSERT OR REPLACE INTO filings
                (ticker, filing_type, filing_date, accession_number, file_path, file_url, download_date)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', (
                ticker,
                filing_type,
                filing_date,
                accession_number,
                file_path,
                file_url,
                datetime.now().strftime('%Y-%m-%d')
            ))
            conn.commit()
        except Exception as e:
            print(f"    ⚠ Error saving to database: {e}")
        finally:
            conn.close()

    def download_filings(self, limit_companies):
        """
        Re-download filings with correct document URLs
        """
        print("="*60)
        print("RE-DOWNLOADING SEC FILINGS (CORRECT FORMAT)")
        print("="*60)
        
        reits = self.get_reit_list()
        
        if limit_companies:
            reits = reits.head(limit_companies)
            print(f"\n⚠ Testing with first {limit_companies} companies")
        
        print(f"\nProcessing {len(reits)} REITs\n")
        
        total_downloaded = 0
        total_errors = 0
        
        for idx, row in reits.iterrows():
            ticker = row['ticker']
            cik = row['cik']
            company = row['company_name']
            
            print(f"[{idx+1}/{len(reits)}] {ticker} - {company}")
            
            # Get most recent 2 10-K filings
            filings_10k = self.get_company_filings_metadata(cik, '10-K', 2)
            filings_10q = self.get_company_filings_metadata(cik, '10-Q', 2)

            for filing in filings_10k:
                if filing['filing_date'] < '2025-01-01':
                    continue
                
                acc_num = filing['accession_number']
                filing_date = filing['filing_date']
                print(f"  → {filing_date} - {acc_num}")
                
                # Get actual document URL
                doc_url = self.get_filing_documents(acc_num, cik)
                
                if doc_url:
                    # Download the document
                    file_path = self.download_filing(
                        doc_url, ticker, acc_num, '10-K'
                    )
                    
                    if file_path:
                        print(f"    ✓ Downloaded to {Path(file_path).name}")
                        # Save to database with file_url
                        self.save_filing_to_db(ticker, '10-K', filing_date, acc_num, file_path, doc_url)
                        total_downloaded += 1
                    else:
                        total_errors += 1
                        print(f"    ✗ Download failed")
                else:
                    total_errors += 1
                    print(f"    ✗ Could not find document URL")
            
            for filing in filings_10q:
                if filing['filing_date'] < '2025-01-01':
                    continue
                
                acc_num = filing['accession_number']
                filing_date = filing['filing_date']
                print(f"  → {filing_date} - {acc_num}")
                
                # Get actual document URL
                doc_url = self.get_filing_documents(acc_num, cik)
                
                if doc_url:
                    # Download the document
                    file_path = self.download_filing(
                        doc_url, ticker, acc_num, '10-Q'
                    )
                    
                    if file_path:
                        print(f"    ✓ Downloaded to {Path(file_path).name}")
                        # Save to database with file_url
                        self.save_filing_to_db(ticker, '10-Q', filing_date, acc_num, file_path, doc_url)
                        total_downloaded += 1
                    else:
                        total_errors += 1
                        print(f"    ✗ Download failed")
                else:
                    total_errors += 1
                    print(f"    ✗ Could not find document URL")

            print()
        
        print("="*60)
        print("DOWNLOAD COMPLETE")
        print("="*60)
        print(f"Downloaded: {total_downloaded}")
        print(f"Errors: {total_errors}")
        
        return total_downloaded, total_errors

if __name__ == "__main__":
    downloader = SECFilingDownloader()
    
    # Test with 5 companies first
    downloader.download_filings(limit_companies=None)

RE-DOWNLOADING SEC FILINGS (CORRECT FORMAT)

Processing 71 REITs

[1/71] PLYM - Plymouth Industrial REIT, Inc.
  → 2025-03-03 - 0001171520-25-000056
    ✓ Downloaded to 000117152025000056_10-K.html
  → 2025-11-10 - 0001193125-25-274281
    ✓ Downloaded to 000119312525274281_10-Q.html
  → 2025-08-06 - 0000950170-25-104096
  ⚠ Error getting document URL: 503 Server Error: Service Unavailable for url: https://www.sec.gov/cgi-bin/viewer?action=view&cik=0001515816&accession_number=0000950170-25-104096&xbrl_type=v
    ✗ Could not find document URL
  → 2025-05-01 - 0001171520-25-000173
    ✓ Downloaded to 000117152025000173_10-Q.html

[2/71] MFA - MFA FINANCIAL, INC.
  → 2025-02-20 - 0001055160-25-000004
    ✓ Downloaded to 000105516025000004_10-K.html
  → 2025-11-06 - 0001055160-25-000018
    ✓ Downloaded to 000105516025000018_10-Q.html
  → 2025-08-06 - 0001055160-25-000013
    ✓ Downloaded to 000105516025000013_10-Q.html
  → 2025-05-06 - 0001055160-25-000007
    ✓ Downloaded to 000105516025

In [22]:
import sqlite3
import pandas as pd
from pathlib import Path
from bs4 import BeautifulSoup
import re
from datetime import datetime
import requests
import time
import json

class FinancialDataExtractor:
    def __init__(self, db_path='version2/data/database/reit_scanner.db'):
        self.db_path = db_path
        self.headers = {
            'User-Agent': 'REIT Scanner tennesseesamwright999@gmail.com',
        }
        
    def get_reit_list(self):
        """Get list of REITs with CIK numbers"""
        conn = sqlite3.connect(self.db_path)
        df = pd.read_sql("SELECT ticker, cik, company_name FROM reits WHERE is_active = 1", conn)
        conn.close()
        return df
    
    def get_company_facts(self, cik):
        """
        Get structured financial data from SEC's Company Facts API
        This returns XBRL data in JSON format - much more reliable!
        """
        cik_clean = str(cik).replace('-', '').zfill(10)
        url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik_clean}.json"
        
        try:
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            time.sleep(0.1)  # Be nice to SEC
            
            data = response.json()
            return data
            
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 404:
                print(f"  ⚠ No XBRL data available for CIK {cik}")
            else:
                print(f"  ⚠ Error fetching company facts: {e}")
            return None
        except Exception as e:
            print(f"  ⚠ Error: {e}")
            return None
    
    def extract_financial_metrics(self, facts_data):
        """
        Extract key financial metrics from SEC Company Facts JSON
        
        The structure is:
        facts['dei'] - document and entity info
        facts['us-gaap'] - US GAAP financial metrics
        """
        if not facts_data or 'facts' not in facts_data:
            return []
        
        metrics_map = {
            # Balance Sheet
            'Assets': 'total_assets',
            'LongTermDebt': 'long_term_debt',
            'DebtCurrent': 'current_debt',
            'Liabilities': 'total_liabilities',
            'StockholdersEquity': 'shareholders_equity',
            'CashAndCashEquivalentsAtCarryingValue': 'cash_and_equivalents',
            
            # Income Statement
            'Revenues': 'total_revenue',
            'OperatingIncomeLoss': 'operating_income',
            'InterestExpense': 'interest_expense',
            'NetIncomeLoss': 'net_income',
            
            # Cash Flow
            'NetCashProvidedByUsedInOperatingActivities': 'operating_cash_flow',
        }
        
        us_gaap = facts_data['facts'].get('us-gaap', {})
        
        # Collect all data points
        all_metrics = []
        
        for gaap_tag, metric_name in metrics_map.items():
            if gaap_tag not in us_gaap:
                continue
            
            metric_data = us_gaap[gaap_tag]
            
            # Get the 10-K and 10-Q filings data
            units = metric_data.get('units', {})
            
            # Most financial metrics are in USD
            usd_data = units.get('USD', [])
            
            for data_point in usd_data:
                # Only get annual (10-K) and quarterly (10-Q) data
                form = data_point.get('form')
                if form not in ['10-K', '10-Q']:
                    continue
                
                all_metrics.append({
                    'metric_name': metric_name,
                    'value': data_point.get('val'),
                    'period_end': data_point.get('end'),
                    'filing_date': data_point.get('filed'),
                    'form': form,
                    'fiscal_year': data_point.get('fy'),
                    'fiscal_period': data_point.get('fp'),
                })
        
        return all_metrics
    
    def save_metrics_to_db(self, ticker, cik, metrics):
        """Save extracted metrics to database"""
        if not metrics:
            return 0
        
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # Group metrics by period
        periods = {}
        for metric in metrics:
            period_key = (metric['period_end'], metric['form'])
            if period_key not in periods:
                periods[period_key] = {
                    'period_end_date': metric['period_end'],
                    'filing_form': metric['form'],
                    'filing_date': metric['filing_date'],
                    'fiscal_year': metric['fiscal_year'],
                    'fiscal_period': metric['fiscal_period'],
                }
            
            periods[period_key][metric['metric_name']] = metric['value']
        
        # Insert each period
        saved = 0
        for period_data in periods.values():
            try:
                cursor.execute('''
                    INSERT OR REPLACE INTO financials 
                    (ticker, period_end_date, 
                     total_assets, long_term_debt, current_debt,
                     total_liabilities, shareholders_equity, cash_and_equivalents,
                     total_revenue, operating_income, interest_expense, net_income,
                     operating_cash_flow, extraction_date)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', (
                    ticker,
                    period_data['period_end_date'],
                    period_data.get('total_assets'),
                    period_data.get('long_term_debt'),
                    period_data.get('current_debt'),
                    period_data.get('total_liabilities'),
                    period_data.get('shareholders_equity'),
                    period_data.get('cash_and_equivalents'),
                    period_data.get('total_revenue'),
                    period_data.get('operating_income'),
                    period_data.get('interest_expense'),
                    period_data.get('net_income'),
                    period_data.get('operating_cash_flow'),
                    datetime.now().strftime('%Y-%m-%d')
                ))
                saved += 1
            except Exception as e:
                print(f"    ⚠ Error saving period {period_data['period_end_date']}: {e}")
                continue
        
        conn.commit()
        conn.close()
        
        return saved
    
    def process_all_reits(self, limit=None):
        """
        Main method to extract financial data for all REITs
        """
        print("="*60)
        print("EXTRACTING FINANCIAL DATA VIA SEC API")
        print("="*60)
        
        reits = self.get_reit_list()
        
        if limit:
            reits = reits.head(limit)
            print(f"\n⚠ Processing only first {limit} REITs for testing")
        
        print(f"\nProcessing {len(reits)} REITs\n")
        
        successful = 0
        failed = 0
        total_periods = 0
        
        for idx, row in reits.iterrows():
            ticker = row['ticker']
            cik = row['cik']
            company = row['company_name']
            
            print(f"[{idx+1}/{len(reits)}] {ticker} - {company}")
            
            # Get structured financial data from SEC API
            print("  Fetching company facts from SEC API...")
            facts = self.get_company_facts(cik)
            
            if facts:
                metrics = self.extract_financial_metrics(facts)
                
                if metrics:
                    saved = self.save_metrics_to_db(ticker, cik, metrics)
                    print(f"  ✓ Extracted {len(metrics)} data points across {saved} periods")
                    total_periods += saved
                    successful += 1
                else:
                    print(f"  ⚠ No financial metrics found in XBRL data")
                    failed += 1
            else:
                failed += 1
            
            print()
        
        print("="*60)
        print("EXTRACTION COMPLETE")
        print("="*60)
        print(f"Successfully processed: {successful}/{len(reits)}")
        print(f"Failed: {failed}/{len(reits)}")
        print(f"Total financial periods saved: {total_periods}")
        print(f"Success rate: {successful/len(reits)*100:.1f}%")
        
        return successful, failed


if __name__ == "__main__":
    extractor = FinancialDataExtractor()
    extractor.process_all_reits()

EXTRACTING FINANCIAL DATA VIA SEC API

Processing 71 REITs

[1/71] PLYM - Plymouth Industrial REIT, Inc.
  Fetching company facts from SEC API...
  ✓ Extracted 1004 data points across 75 periods

[2/71] MFA - MFA FINANCIAL, INC.
  Fetching company facts from SEC API...
  ✓ Extracted 1344 data points across 118 periods

[3/71] GMRE - Global Medical REIT Inc.
  Fetching company facts from SEC API...
  ✓ Extracted 1027 data points across 95 periods

[4/71] AIV - APARTMENT INVESTMENT & MANAGEMENT CO
  Fetching company facts from SEC API...
  ✓ Extracted 1655 data points across 117 periods

[5/71] FBRT - Franklin BSP Realty Trust, Inc.
  Fetching company facts from SEC API...
  ✓ Extracted 1238 data points across 82 periods

[6/71] BRSP - BrightSpire Capital, Inc.
  Fetching company facts from SEC API...
  ✓ Extracted 731 data points across 46 periods

[7/71] STRW - Strawberry Fields REIT, Inc.
  Fetching company facts from SEC API...
  ✓ Extracted 297 data points across 23 periods

[8/71] 

In [23]:
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime

class RatioCalculator:
    def __init__(self, db_path='version2/data/database/reit_scanner.db'):
        self.db_path = db_path
    
    def get_financial_data(self):
        """Get all financial data from database"""
        conn = sqlite3.connect(self.db_path)
        
        query = """
            SELECT 
                ticker,
                period_end_date,
                total_assets,
                long_term_debt,
                current_debt,
                total_liabilities,
                shareholders_equity,
                cash_and_equivalents,
                total_revenue,
                operating_income,
                interest_expense,
                net_income,
                operating_cash_flow
            FROM financials
            WHERE total_assets IS NOT NULL
            ORDER BY ticker, period_end_date DESC
        """
        
        df = pd.read_sql(query, conn)
        conn.close()
        
        return df
    
    def calculate_all_ratios(self, df):
        """Calculate leverage and liquidity ratios"""
        
        # Calculate total debt (long-term + current)
        df['total_debt'] = df['long_term_debt'].fillna(0) + df['current_debt'].fillna(0)
        
        # Replace zeros with NaN to avoid division errors
        df['total_assets'] = df['total_assets'].replace(0, np.nan)
        df['shareholders_equity'] = df['shareholders_equity'].replace(0, np.nan)
        df['interest_expense'] = df['interest_expense'].replace(0, np.nan)
        df['total_revenue'] = df['total_revenue'].replace(0, np.nan)
        
        # Leverage Ratios
        df['debt_to_assets'] = df['total_debt'] / df['total_assets']
        df['debt_to_equity'] = df['total_debt'] / df['shareholders_equity']
        
        # Coverage Ratios
        # Interest Coverage = Operating Income / Interest Expense
        df['interest_coverage'] = df['operating_income'] / df['interest_expense']
        
        # Liquidity Ratios
        df['cash_to_debt'] = df['cash_and_equivalents'] / df['total_debt']
        
        # Additional useful metrics
        df['equity_ratio'] = df['shareholders_equity'] / df['total_assets']
        df['leverage_ratio'] = df['total_assets'] / df['shareholders_equity']
        
        return df
    
    def get_most_recent_by_ticker(self, df):
        """Get most recent financial period for each REIT"""
        # Sort by ticker and date, then take the first (most recent) for each ticker
        df_sorted = df.sort_values(['ticker', 'period_end_date'], ascending=[True, False])
        most_recent = df_sorted.groupby('ticker').first().reset_index()
        
        return most_recent
    
    def calculate_leverage_score(self, row):
        """
        Calculate leverage score (0-100, higher = more risk)
        Based on debt/assets, debt/equity, and interest coverage
        """
        score = 0
        
        # Debt to Assets (weight: 30 points)
        debt_assets = row['debt_to_assets']
        if pd.notna(debt_assets):
            if debt_assets > 0.65:  # Very high leverage
                score += 30
            elif debt_assets > 0.55:
                score += 25
            elif debt_assets > 0.45:
                score += 15
            elif debt_assets > 0.35:
                score += 5
        
        # Debt to Equity (weight: 25 points)
        debt_equity = row['debt_to_equity']
        if pd.notna(debt_equity):
            if debt_equity > 3.0:  # Very high
                score += 25
            elif debt_equity > 2.0:
                score += 20
            elif debt_equity > 1.5:
                score += 10
            elif debt_equity > 1.0:
                score += 5
        
        # Interest Coverage (weight: 45 points) - MOST IMPORTANT
        int_cov = row['interest_coverage']
        if pd.notna(int_cov):
            if int_cov < 1.0:  # Not covering interest!
                score += 45
            elif int_cov < 1.5:  # Barely covering
                score += 35
            elif int_cov < 2.0:
                score += 25
            elif int_cov < 2.5:
                score += 15
            elif int_cov < 3.0:
                score += 5
        else:
            # No interest expense data - could be concerning
            score += 10
        
        return min(score, 100)  # Cap at 100
    
    def calculate_liquidity_score(self, row):
        """
        Calculate liquidity score (0-100, higher = more risk)
        Based on cash position relative to debt
        """
        score = 0
        
        # Cash to Debt ratio (lower = worse)
        cash_debt = row['cash_to_debt']
        if pd.notna(cash_debt):
            if cash_debt < 0.02:  # Less than 2% cash coverage
                score += 50
            elif cash_debt < 0.05:  # Less than 5%
                score += 35
            elif cash_debt < 0.10:  # Less than 10%
                score += 20
            elif cash_debt < 0.15:
                score += 10
        else:
            score += 25  # No data is concerning
        
        # Equity Ratio (lower = worse)
        equity_ratio = row['equity_ratio']
        if pd.notna(equity_ratio):
            if equity_ratio < 0.20:  # Less than 20% equity
                score += 50
            elif equity_ratio < 0.30:
                score += 30
            elif equity_ratio < 0.40:
                score += 15
            elif equity_ratio < 0.50:
                score += 5
        
        return min(score, 100)
    
    def process_all_reits(self):
        """Main method to calculate ratios and scores"""
        print("="*60)
        print("CALCULATING FINANCIAL RATIOS & SCORES")
        print("="*60)
        
        # Get data
        print("\nLoading financial data...")
        df = self.get_financial_data()
        print(f"Loaded {len(df)} financial records for {df['ticker'].nunique()} REITs")
        
        # Calculate ratios
        print("\nCalculating leverage and liquidity ratios...")
        df_with_ratios = self.calculate_all_ratios(df)
        
        # Get most recent period for each REIT
        print("Getting most recent data for each REIT...")
        most_recent = self.get_most_recent_by_ticker(df_with_ratios)
        print(f"Processing {len(most_recent)} REITs")
        
        # Calculate scores
        print("\nCalculating risk scores...")
        most_recent['leverage_score'] = most_recent.apply(self.calculate_leverage_score, axis=1)
        most_recent['liquidity_score'] = most_recent.apply(self.calculate_liquidity_score, axis=1)
        
        # Composite score (we'll add NLP distress score later)
        # For now: 50% leverage, 50% liquidity
        most_recent['financial_score'] = (
            most_recent['leverage_score'] * 0.5 +
            most_recent['liquidity_score'] * 0.5
        )
        
        # Rank REITs
        most_recent['rank'] = most_recent['financial_score'].rank(ascending=False, method='dense').astype(int)
        
        # Save to database
        print("\nSaving scores to database...")
        self.save_scores(most_recent)
        
        # Display results
        print("\n" + "="*60)
        print("TOP 20 MOST AT-RISK REITs (by financial metrics)")
        print("="*60)
        
        top_risk = most_recent.nlargest(20, 'financial_score')[[
            'ticker', 'period_end_date', 'financial_score', 'leverage_score', 
            'liquidity_score', 'debt_to_assets', 'interest_coverage', 'cash_to_debt'
        ]]
        
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', None)
        pd.set_option('display.max_colwidth', None)
        print(top_risk.to_string(index=False))
        
        # Save to CSV
        output_path = 'version2/data/processed/reit_risk_scores.csv'
        most_recent.to_csv(output_path, index=False)
        print(f"\n✓ Saved full results to {output_path}")
        
        # Summary statistics
        print("\n" + "="*60)
        print("RISK SCORE DISTRIBUTION")
        print("="*60)
        print(most_recent['financial_score'].describe())
        
        print("\n" + "="*60)
        print("KEY METRICS SUMMARY")
        print("="*60)
        print(f"REITs with Debt/Assets > 60%: {(most_recent['debt_to_assets'] > 0.6).sum()}")
        print(f"REITs with Interest Coverage < 2.0x: {(most_recent['interest_coverage'] < 2.0).sum()}")
        print(f"REITs with Cash/Debt < 5%: {(most_recent['cash_to_debt'] < 0.05).sum()}")
        
        return most_recent
    
    def save_scores(self, df):
        """Save scores to database"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        for _, row in df.iterrows():
            cursor.execute('''
                INSERT OR REPLACE INTO reit_scores
                (ticker, analysis_date, period_end_date, 
                 leverage_score, liquidity_score, final_score, rank)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', (
                row['ticker'],
                datetime.now().strftime('%Y-%m-%d'),
                row['period_end_date'],
                row['leverage_score'],
                row['liquidity_score'],
                row['financial_score'],
                row['rank']
            ))
        
        conn.commit()
        conn.close()


if __name__ == "__main__":
    calculator = RatioCalculator()
    results = calculator.process_all_reits()

CALCULATING FINANCIAL RATIOS & SCORES

Loading financial data...
Loaded 10887 financial records for 71 REITs

Calculating leverage and liquidity ratios...
Getting most recent data for each REIT...
Processing 71 REITs

Calculating risk scores...

Saving scores to database...

TOP 20 MOST AT-RISK REITs (by financial metrics)
ticker period_end_date  financial_score  leverage_score  liquidity_score  debt_to_assets  interest_coverage  cash_to_debt
  MDRR      2025-09-30             92.5             100               85        0.690647           0.574280      0.029659
  SOHO      2025-09-30             92.5             100               85        0.779108           0.092408      0.029430
  STRW      2025-09-30             82.5              80               85        0.884988           1.652522      0.025412
   ACR      2025-09-30             80.0              95               65        0.706957           0.638497      0.034281
   AHT      2025-09-30             80.0              75          

In [27]:
import sqlite3
import pandas as pd
from pathlib import Path
from bs4 import BeautifulSoup
import re

class TextSectionExtractor:
    def __init__(self):
        self.filings_dir = Path('version2/data/raw/filings')
    
    def get_all_10k_files(self):
        """Get all 10-K and 10-Q files from the new download directory"""
        all_files = []
        
        for ticker_folder in self.filings_dir.iterdir():
            if ticker_folder.is_dir():
                ticker = ticker_folder.name
                for file in ticker_folder.glob('*_10-[KQ].html'):
                    all_files.append({
                        'ticker': ticker,
                        'file_path': str(file),
                        'filename': file.name
                    })
        
        return pd.DataFrame(all_files)
    
    def extract_text_from_html(self, file_path):
        """Extract clean text from HTML filing"""
        try:
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                content = f.read()
            
            soup = BeautifulSoup(content, 'html.parser')
            
            # Remove scripts, styles
            for element in soup(['script', 'style']):
                element.decompose()
            
            text = soup.get_text(separator=' ', strip=True)
            return text
            
        except Exception as e:
            return ""
    
    def extract_sections(self, text):
        """Extract MD&A and Risk Factors"""
        mda = None
        risks = None
        
        # MD&A patterns
        mda_patterns = [
            r"ITEM\s*7[\s\.:\-–—]*MANAGEMENT['\']?S?\s+DISCUSSION",
            r"Item\s*7[\s\.:\-–—]*Management['\']?s?\s+Discussion",
        ]
        
        for pattern in mda_patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                start = match.start()
                # Get next 30k chars
                mda = text[start:start+30000]
                break
        
        # Risk Factors patterns
        risk_patterns = [
            r"ITEM\s*1A[\s\.:\-–—]*RISK\s+FACTORS",
            r"Item\s*1A[\s\.:\-–—]*Risk\s+Factors",
        ]
        
        for pattern in risk_patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                start = match.start()
                risks = text[start:start+30000]
                break
        
        return mda, risks
    
    def process_all_filings(self, limit_reits=None):
        """Extract text from all 10-K and 10-Q filings"""
        print("="*60)
        print("EXTRACTING TEXT SECTIONS")
        print("="*60)
        
        files_df = self.get_all_10k_files()
        print(f"\nFound {len(files_df)} 10-K and 10-Q files across {files_df['ticker'].nunique()} REITs")
        

        if limit_reits:
            tickers = files_df['ticker'].unique()[:limit_reits]
            files_df = files_df[files_df['ticker'].isin(tickers)]
            print(f"\n⚠ Processing only {limit_reits} REITs for testing")
        
        print(f"Processing {len(files_df)} files\n")
        
        mda_count = 0
        risk_count = 0
        
        for idx, row in files_df.iterrows():
            ticker = row['ticker']
            file_path = row['file_path']
            
            if idx % 20 == 0:
                print(f"[{idx}/{len(files_df)}] {ticker}...")
            
            text = self.extract_text_from_html(file_path)
            if not text or len(text) < 5000:
                continue
            
            mda, risks = self.extract_sections(text)
            
            if mda or risks:
                text_dir = Path(f'version2/data/processed/text_sections/{ticker}')
                text_dir.mkdir(parents=True, exist_ok=True)
                
                file_id = Path(file_path).stem.split('_')[0]
                
                if mda and len(mda) > 1000:
                    with open(text_dir / f'{file_id}_mda.txt', 'w', encoding='utf-8') as f:
                        f.write(mda)
                    mda_count += 1
                
                if risks and len(risks) > 1000:
                    with open(text_dir / f'{file_id}_risks.txt', 'w', encoding='utf-8') as f:
                        f.write(risks)
                    risk_count += 1
        
        print("\n" + "="*60)
        print("TEXT EXTRACTION COMPLETE")
        print("="*60)
        print(f"MD&A sections extracted: {mda_count}")
        print(f"Risk sections extracted: {risk_count}")
        
        text_dir = Path('version2/data/processed/text_sections')
        if text_dir.exists():
            reits = [d.name for d in text_dir.iterdir() if d.is_dir()]
            print(f"REITs with text data: {len(reits)}")


if __name__ == "__main__":
    extractor = TextSectionExtractor()
    extractor.process_all_filings()

EXTRACTING TEXT SECTIONS

Found 285 10-K and 10-Q files across 71 REITs
Processing 285 files

[0/285] ACR...
[20/285] AIV...
[40/285] BRSP...
[60/285] CLDT...
[80/285] EARN...
[100/285] FSP...
[120/285] IHT...
[140/285] LAND...
[160/285] MFA...
[180/285] NYC...
[200/285] PLYM...
[220/285] RWT...
[240/285] SOHO...
[260/285] SUNS...
[280/285] WHLR...

TEXT EXTRACTION COMPLETE
MD&A sections extracted: 24
Risk sections extracted: 261
REITs with text data: 71


In [28]:
import sqlite3
import pandas as pd
from pathlib import Path
import json
from datetime import datetime
import time
import os

from openai import OpenAI


# SET YOUR UF NAVIGATOR / OPENAI API KEY HERE
os.environ["OPENAI_API_KEY"] = "sk-tbJgticiapFJTC4vMiHnhQ"


class NLPDistressAnalyzer:
    def __init__(self, db_path='version2/data/database/reit_scanner.db'):
        self.db_path = db_path

        # UF Navigator / LiteLLM OpenAI-compatible client
        self.client = OpenAI(
            api_key=os.environ["OPENAI_API_KEY"],
            base_url="https://api.ai.it.ufl.edu"
        )

    def get_reits_with_text_sections(self):
        text_dir = Path('version2/data/processed/text_sections')
        if not text_dir.exists():
            return []

        return [
            folder.name
            for folder in text_dir.iterdir()
            if folder.is_dir() and any(folder.glob('*'))
        ]

    def analyze_filing_text(self, ticker, mda_text=None, risk_text=None):
        if not mda_text and not risk_text:
            return None

        combined_text = ""
        if mda_text:
            combined_text += f"MD&A SECTION:\n{mda_text[:8000]}\n\n"
        if risk_text:
            combined_text += f"RISK FACTORS:\n{risk_text[:7000]}"

        prompt = f"""
You are a financial analyst specializing in REIT distress analysis.

Analyze the following excerpts from {ticker}'s 10-K filing for signs of financial distress or forced asset sales.

Look for:
1. Covenant violations or risks
2. Liquidity concerns
3. Forced asset sales
4. Debt maturity pressures
5. Operational deterioration
6. Going concern warnings

Filing Text:
{combined_text}

Respond ONLY with valid JSON (no markdown):

{{
  "distress_score": <0-10>,
  "confidence": "<high|medium|low>",
  "key_flags": [
    {{
      "category": "<covenant_risk|liquidity|asset_sale|debt_maturity|operational|going_concern>",
      "severity": "<critical|high|medium|low>",
      "description": "<brief description>",
      "supporting_quote": "<relevant excerpt>"
    }}
  ],
  "summary": "<2-3 sentence summary>"
}}
"""

        try:
            response = self.client.chat.completions.create(
                model="gpt-oss-120b",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.2
            )

            response_text = response.choices[0].message.content
            response_text = response_text.replace("```json", "").replace("```", "").strip()

            return json.loads(response_text)

        except Exception as e:
            print(f"  ⚠ Error analyzing {ticker}: {e}")
            return None

    def save_nlp_analysis(self, ticker, filing_id, analysis):
        if not analysis:
            return

        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()


        cursor.execute("""
            INSERT INTO nlp_analysis
            (ticker, filing_id, distress_score, sentiment_score,
             analysis_date, model_used)
            VALUES (?, ?, ?, ?, ?, ?)
        """, (
            ticker,
            filing_id,
            analysis.get('distress_score'),
            None,
            datetime.now().strftime('%Y-%m-%d'),
            'gpt-oss-120b'
        ))

        for flag in analysis.get('key_flags', []):
            cursor.execute("""
                INSERT INTO distress_flags
                (ticker, filing_id, flag_category, severity,
                 description, supporting_quote, detected_date)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, (
                ticker,
                filing_id,
                flag.get('category'),
                flag.get('severity'),
                flag.get('description'),
                flag.get('supporting_quote'),
                datetime.now().strftime('%Y-%m-%d')
            ))

        conn.commit()
        conn.close()

    def process_all_reits(self, limit=None):
        print("=" * 60)
        print("NLP DISTRESS ANALYSIS (UF NAVIGATOR)")
        print("=" * 60)

        reits = self.get_reits_with_text_sections()

        if limit:
            reits = reits[:limit]
            print(f"\n⚠ Running in test mode: {limit} REITs\n")

        for idx, ticker in enumerate(reits, 1):
            print(f"[{idx}/{len(reits)}] Analyzing {ticker}")

            text_dir = Path(f'version2/data/processed/text_sections/{ticker}')
            mda_files = sorted(text_dir.glob('*_mda.txt'), reverse=True)
            risk_files = sorted(text_dir.glob('*_risks.txt'), reverse=True)

            mda_text = None
            risk_text = None
            filing_id = None

            if mda_files:
                filing_id = mda_files[0].stem.split('_')[0]
                mda_text = mda_files[0].read_text(encoding='utf-8')

            if risk_files:
                if not filing_id:
                    filing_id = risk_files[0].stem.split('_')[0]
                risk_text = risk_files[0].read_text(encoding='utf-8')

            analysis = self.analyze_filing_text(ticker, mda_text, risk_text)

            if analysis:
                print(f"  Distress Score: {analysis['distress_score']} | Confidence: {analysis['confidence']}")
                print(f"  Flags Found: {len(analysis.get('key_flags', []))}")
                self.save_nlp_analysis(ticker, filing_id, analysis)

            time.sleep(0.5)

        print("\n✅ NLP distress analysis complete")






analyzer = NLPDistressAnalyzer()

analyzer.process_all_reits()

# Full run (later)
# analyzer.process_all_reits()



NLP DISTRESS ANALYSIS (UF NAVIGATOR)
[1/71] Analyzing ACR
  Distress Score: 6.5 | Confidence: medium
  Flags Found: 6
[2/71] Analyzing ACRE
  Distress Score: 3 | Confidence: medium
  Flags Found: 2
[3/71] Analyzing AFCG
  Distress Score: 7 | Confidence: medium
  Flags Found: 5
[4/71] Analyzing AHH
  Distress Score: 6.5 | Confidence: medium
  Flags Found: 6
[5/71] Analyzing AHT
  Distress Score: 8 | Confidence: high
  Flags Found: 6
[6/71] Analyzing AIV
  Distress Score: 7.5 | Confidence: high
  Flags Found: 6
[7/71] Analyzing AOMR
  Distress Score: 6.5 | Confidence: medium
  Flags Found: 6
[8/71] Analyzing BDN
  Distress Score: 5 | Confidence: medium
  Flags Found: 6
[9/71] Analyzing BHM
  Distress Score: 7 | Confidence: medium
  Flags Found: 6
[10/71] Analyzing BHR
  Distress Score: 7 | Confidence: medium
  Flags Found: 6
[11/71] Analyzing BRSP
  Distress Score: 6 | Confidence: medium
  Flags Found: 6
[12/71] Analyzing BRT
  Distress Score: 7 | Confidence: medium
  Flags Found: 4
[13/

In [43]:
db_path = "version2/data/database/reit_scanner.db"
ticker = "NLOP"   # change this

conn = sqlite3.connect(db_path)

ticker_flags = pd.read_sql("""
    SELECT
        *
    FROM reits
""", conn)

conn.close()

ticker_flags

,ticker,company_name,cik,market_cap,sector,reit_type,geography,is_active,date_added,last_updated
0,PLYM,"Plymouth Industrial REIT, Inc.",0001515816,987534080,Real Estate,Equity,US,1,2026-01-13,2026-01-13
1,MFA,"MFA FINANCIAL, INC.",0001055160,989212992,Real Estate,Equity,US,1,2026-01-13,2026-01-13
2,GMRE,Global Medical REIT Inc.,0001533615,960033728,Real Estate,Equity,US,1,2026-01-13,2026-01-13
3,AIV,APARTMENT INVESTMENT & MANAGEMENT CO,0000922864,848316736,Real Estate,Equity,US,1,2026-01-13,2026-01-13
4,FBRT,"Franklin BSP Realty Trust, Inc.",0001562528,813286912,Real Estate,Equity,US,1,2026-01-13,2026-01-13
...,...,...,...,...,...,...,...,...,...,...
66,SQFT,"Presidio Property Trust, Inc.",0001080657,5202343,Real Estate,Equity,US,1,2026-01-13,2026-01-13
67,GIPR,"GENERATION INCOME PROPERTIES, INC.",0001651721,4491688,Real Estate,Equity,US,1,2026-01-13,2026-01-13
68,WHLR,"Wheeler Real Estate Investment Trust, Inc.",0001527541,3217630,Real Estate,Equity,US,1,2026-01-13,2026-01-13
69,CMCT,Creative Media & Community Trust Corp,0000908311,3053603,Real Estate,Equity,US,1,2026-01-13,2026-01-13


In [45]:
db_path = "version2/data/database/reit_scanner.db"
ticker = "NLOP"   # change this

conn = sqlite3.connect(db_path)

ticker_flags = pd.read_sql("""
    SELECT
        d.ticker,
        r.company_name,
        r.market_cap,
        d.filing_id,
        f.filing_type,
        f.filing_date,
        f.file_url,
        d.flag_category,
        d.severity,
        d.description,
        d.supporting_quote
    FROM distress_flags d 
    JOIN filings f ON d.filing_id = f.accession_number
    JOIN reits r ON d.ticker = r.ticker
""", conn)

conn.close()

ticker_flags
with pd.ExcelWriter('version2/data/processed/Version2.xlsx', engine='openpyxl') as writer:
    ticker_flags.to_excel(writer, sheet_name='Version2', index=False)

print(f"✓ Saved {len(ticker_flags)} distress flags to version2/data/processed/Version2.xlsx")


✓ Saved 370 distress flags to version2/data/processed/Version2.xlsx


In [ ]:
import sqlite3
import pandas as pd

db_path = "version2/data/database/reit_scanner.db"
ticker = "NLOP"   # change this

conn = sqlite3.connect(db_path)

ticker_flags = pd.read_sql("""
    SELECT
        ticker,
        period_end_date,
        leverage_score,
        liquidity_score,
        distress_score,
        final_score,
        rank
    FROM reit_scores
    ORDER BY rank
""", conn)

conn.close()

ticker_flags
with pd.ExcelWriter('version2/data/processed/distress_flags.xlsx', engine='openpyxl', mode='a') as writer:
    ticker_flags.to_excel(writer, sheet_name='reit_scores', index=False)

print(f"✓ Saved {len(ticker_flags)} REIT scores to version2/data/processed/distress_flags.xlsx (sheet: reit_scores)")


✓ Saved 218 REIT scores to data/processed/distress_flags.xlsx (sheet: reit_scores)


In [ ]:
db_path = "version2/data/database/reit_scanner.db"
ticker = "NLOP"   # change this

conn = sqlite3.connect(db_path)

ticker_flags = pd.read_sql("""
    SELECT
    ticker,
    company_name,
    market_cap,
    geography
    FROM reits
""", conn)

conn.close()

with pd.ExcelWriter('version2/data/processed/distress_flags.xlsx', engine='openpyxl', mode='a') as writer:
    ticker_flags.to_excel(writer, sheet_name='reits', index=False)

print(f"✓ Saved {len(ticker_flags)} REIT records to version2/data/processed/distress_flags.xlsx (sheet: reits)")

✓ Saved 117 REIT records to data/processed/distress_flags.xlsx (sheet: reits)
